# PDBBind 데이터셋
- 생체 분자의 3차원 구조와 이들의 결합 친화력을 포함
 - 단백질-단백질, 단백질-리간드, 단백질-DNA, DNA-리간드 등
 - (참고) docking: 단백질-리간의 결합 부위를 예측하는 도구
- 단백질 정보 은행 (PDB)에서 수집되었다
 - 잘 정리된 데이터: 핵심 core 데이터만 사용하겠다
- 결합 친화력이 클수록 두 분자가 복합체 형태로 더 많이 
존재한다는 뜻

## 랜덤 포레스트

In [1]:
!pip install deepchem

     |████████████████████████████████| 608 kB 9.8 MB/s 
     |████████████████████████████████| 22.5 MB 87.3 MB/s 


## 그리드 피처기 옵션
- sanitize=True : DeepChem이 탐지한 명백한 오류는 수정
- flatten=True : 각 입력에 대해 1차원 피처 벡터를 출력
- 복셀(voxel) : 픽셀의 3차원 아나로그 값

- splif: Structural Protein-Ligand Interaction Fingerprint

In [31]:
# 데이터 로드
import deepchem as dc
featurizer = dc.feat.RdkitGridFeaturizer(flatten=True, sanitize=True, 
             feature_types=['hbond', 'salt_bridge', 'pi_stack', 'cation_pi', 'ecfp', 'splif'])
pdbbind_tasks, pdbbind_datasets, transformers = dc.molnet.load_pdbbind(featurizer=featurizer, splitter="random", subset="core")
train_dataset, valid_dataset, test_dataset = pdbbind_datasets


In [32]:
train_dataset.X.shape

(154, 1)

In [36]:
train_dataset.y.shape

(154,)

In [33]:

# 랜덤 포레스트 모델 사용

from sklearn.ensemble import RandomForestRegressor
sklearn_model = RandomForestRegressor(n_estimators=100)
model = dc.models.SklearnModel(sklearn_model, model_dir="pdbbind_rf")
model.fit(train_dataset)

# 검증

metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
train_scores = model.evaluate(train_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)
print("Train scores")
print(train_scores)
print("Test scores")
print(test_scores)

Train scores
{'pearson_r2_score': nan}
Test scores
{'pearson_r2_score': nan}


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


## 신경망

In [34]:

n_features = train_dataset.X.shape[1]
model = dc.models.MultitaskRegressor(
        n_tasks=len(pdbbind_tasks),
        n_features=n_features,
        layer_sizes=[2000, 1000],
        dropouts=0.5,
        model_dir="pdbbind_nn",
        learning_rate=0.0003)
model.fit(train_dataset, nb_epoch=50)

# Evaluate it.

metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
train_scores = model.evaluate(train_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)
print("Train scores")
print(train_scores)
print("Test scores")
print(test_scores)

Train scores
{'pearson_r2_score': 0.002704200083891712}
Test scores
{'pearson_r2_score': 0.008446555369604352}
